In [ ]:
from tensorflow.keras import layers
from tensorflow import keras
import tensorflow as tf

import matplotlib.pyplot as plt
import numpy as np

In [ ]:
import tensorflow as tf
INPUT_WIDTH = 128
INPUT_HEIGHT = 128
N_CHANNELS = 12
N_CLASSES = 2

In [ ]:
import uuid

def load_dir(data_dir):
  id_img_map = {}
  fire_sample_map = {}
  for f in os.listdir(data_dir):
    if '_' in f and not 'download_log' in f:
      id = f[:f.index('_')]
      if id in fire_sample_map:
        id_img_map[fire_sample_map[id]].append(f'{data_dir}{f}')
      else:
        fire_sample_map[id] = str(uuid.uuid4())
        id_img_map[fire_sample_map[id]] = [f'{data_dir}{f}']
  return id_img_map

In [ ]:
id_img_map_negative = load_dir('data/TrainingData/AllData/Fire-Negative-5mo/')
id_img_map_positive = load_dir('data/TrainingData/AllData/Fire-Positive-5mo/')

In [ ]:
from sklearn.model_selection import train_test_split

SIZE = 2769
all_ids_negative = list(zip(id_img_map_negative.keys(), [0]*SIZE))
all_ids_positive = list(zip(id_img_map_positive.keys(), [1]*SIZE))
all_ids = all_ids_negative + all_ids_positive
ids_train, ids_test = train_test_split(all_ids, test_size=0.2, random_state=42)
ids_train, ids_val = train_test_split(ids_train, test_size=0.25, random_state=42)

In [ ]:
def batch(iterable, n=1):
    l = len(iterable)
    for ndx in range(0, l, n):
        yield iterable[ndx:min(ndx + n, l)]

BATCH_SIZE = 32
train_batch_ids = list(batch(ids_train, BATCH_SIZE))
val_batch_ids = list(batch(ids_val, BATCH_SIZE))
test_batch_ids = list(batch(ids_test, BATCH_SIZE))

In [ ]:
def scale_resize_image(image):
    image = tf.image.convert_image_dtype(image, tf.float32) # equivalent to dividing image pixels by 255
    image = tf.image.resize(image, (INPUT_WIDTH, INPUT_HEIGHT)) # Resizing the image to 224x224 dimention
    return image

In [ ]:
def get_image_data(id_img_map, sample_id):
  images = []
  fs = id_img_map[sample_id]
  for i,f in enumerate(fs[::-1]): # Reverse reversed time
    image = np.load(f)
    image_resized = scale_resize_image(image)
    images.append(image_resized)
  input = np.array(images)
  return input

In [ ]:
from tqdm import tqdm

def get_batch_data(batch):
  X = []
  y = []
  for sample_id, label in batch:
    id_img_map = id_img_map_positive if label==1 else id_img_map_negative
    x = get_image_data(id_img_map, sample_id)
    X.append(x)
    y.append(label)
  X = np.array(X)
  y = np.array(y)
  return X, y

100%|██████████| 104/104 [02:46<00:00,  1.60s/it]


In [ ]:
def save_dataset(dataset, name):
  for i, batch in enumerate(dataset):
    X,y = batch
    with open(f'data/TrainingData/{name}/{i}_X.npy', 'wb') as f:
      np.save(f, X)
    with open(f'data/TrainingData/{name}/{i}_y.npy', 'wb') as f:
      np.save(f, y)

In [ ]:
train_data = []
for batch in tqdm(train_batch_ids):
  X, y = get_batch_data(batch)
  train_data += [[X,y]]

In [ ]:
save_dataset(train_data, 'train')

In [ ]:
val_data = []
for batch in tqdm(val_batch_ids):
  X, y = get_batch_data(batch)
  val_data += [[X,y]]

In [ ]:
save_dataset(val_data, 'val')

In [ ]:
test_data = []
for batch in tqdm(test_batch_ids):
  X, y = get_batch_data(batch)
  test_data += [[X,y]]

100%|██████████| 35/35 [14:33<00:00, 24.97s/it]


In [ ]:
save_dataset(test_data, 'test')

In [ ]:
class My_Custom_Generator(keras.utils.Sequence) :
  
  def __init__(self, image_filenames, labels, batch_size) :
    self.image_filenames = image_filenames
    self.labels = labels
    self.batch_size = batch_size
    
    
  def __len__(self) :
    return (np.ceil(len(self.image_filenames) / float(self.batch_size))).astype(np.int)
  
  
  def __getitem__(self, idx) :
    batch_x = self.image_filenames[idx * self.batch_size : (idx+1) * self.batch_size]
    batch_y = self.labels[idx * self.batch_size : (idx+1) * self.batch_size]
    
    return np.array([
            resize(imread('/content/all_images/' + str(file_name)), (80, 80, 3))
               for file_name in batch_x])/255.0, np.array(batch_y)